In [1]:
import random
from string import ascii_uppercase
from pprint import pprint

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [4]:
base_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in base_rotors.keys():
    base_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in base_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [42]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [21]:
reflectors = {'Brf': {k:v for k,v in zip(entry, breflector)}, 'Crf': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [48]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', second_rotor='II', third_rotor='III'):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
#     print(entry_pos)
    fst_out = usedict[first_rotor][entry_pos]
#     print(fst_out)
    ch1 = entry[fst_out]
    print('first rotor:  ', ch, '-->', ch1)
    scd_out = usedict[second_rotor][fst_out]
#     print(scd_out)
    ch2 = entry[scd_out]
    print('second rotor: ', ch1, '-->', ch2)
    thd_out = usedict[third_rotor][scd_out]
#     print(thd_out)
    ch3 = entry[thd_out]
    print('third rotor:  ', ch2, '-->', ch3)
    
    return ch3
    
    
once_thru_scramble('d')

first rotor:   D --> F
second rotor:  F --> I
third rotor:   I --> R


'R'

In [53]:
def full_scramble(in_ch, left_rotor='I', middle_rotor='II', right_rotor='III', rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    forward_run = once_thru_scramble(in_ch, 'forward', left_rotor, middle_rotor, right_rotor)
    
    mirrored = reflectors[rflector][forward_run]
    print('reflected:    ', forward_run, '-->', mirrored)
    
    back_run = once_thru_scramble(mirrored, 'back', right_rotor, middle_rotor, left_rotor)
    
    return back_run

full_scramble('a')

first rotor:   A --> E
second rotor:  E --> S
third rotor:   S --> G
reflected:     G --> L
first rotor:   L --> F
second rotor:  F --> W
third rotor:   W --> N


'N'